In [3]:
import torch
trajectories = torch.load("outputs/2025-12-05/17-31-24/results.pt")

In [4]:
coord_std = torch.load("data/qm9/processed/distributions.pt")['coordinate_std']

In [5]:
from rdkit.Chem import AllChem as Chem

In [6]:
tokens = ["Si", # mask
"P", # death
"C",
"F",
"H",
"N",
"O",
]

In [7]:
from chemflow.repr import tensors_to_rdkit_mol

/cluster/project/jorner/zivkovic/chemflow/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
def split_by_batch(x, batch_id):
    # x: (N_nodes, F)
    # batch_id: (N_nodes,)
    num_graphs = int(batch_id.max().item()) + 1

    return [
        x[batch_id == g]      # tensor for graph g
        for g in range(num_graphs)
    ]

In [9]:
traj = trajectories[0]

In [10]:
traj.keys()

dict_keys(['final_coord', 'final_atom_types', 'final_batch_index', 'xt_trajectory', 'at_trajectory', 'batch_id_trajectory', 'x_pred_trajectory', 'a_pred_trajectory'])

In [11]:
atom_types = [split_by_batch(at, traj['batch_id_trajectory'][i]) for i, at in enumerate(traj['at_trajectory'])]
coord = [split_by_batch(xt, traj['batch_id_trajectory'][i]) for i, xt in enumerate(traj['xt_trajectory'])]
a_pred = [split_by_batch(at, traj['batch_id_trajectory'][i]) for i, at in enumerate(traj['a_pred_trajectory'])]
x_pred = [split_by_batch(xt, traj['batch_id_trajectory'][i]) for i, xt in enumerate(traj['x_pred_trajectory'])]

In [18]:
with Chem.SDWriter('test.sdf') as w:    
    for a, c in zip(atom_types, coord):
        mol = tensors_to_rdkit_mol(a[1], c[1], tokens, infer_bonds=False)
        w.write(mol)
with Chem.SDWriter('test_pred.sdf') as w:    
    for a, c in zip(a_pred, x_pred):
        mol = tensors_to_rdkit_mol(torch.argmax(a[1], dim=-1), c[1], tokens, infer_bonds=False)
        w.write(mol)

In [27]:
mol = tensors_to_rdkit_mol(final_atom_types[150], final_coord[150]*coord_std, tokens, infer_bonds=False)

In [17]:
import py3Dmol

In [28]:
view = py3Dmol.view(
    data=Chem.MolToMolBlock(mol),  # Convert the RDKit molecule for py3Dmol
    style={"stick": {}, "sphere": {"scale": 0.3}}
)
view.zoomTo()

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [12]:
trajectories[0].keys()

dict_keys(['final_coord', 'final_atom_types', 'final_batch_index', 'xt_trajectory', 'at_trajectory', 'batch_id_trajectory'])